In [9]:
%load_ext autoreload
%autoreload 2
import numpy as np
import deepSI as dsi
import torch

## Generate dataset ## 
np.random.seed(0)
ulist = np.random.randn(10_000) #input sequence
x = [0, 0] #initial state
ylist = [] #output sequence
for uk in ulist:
    ylist.append(x[1]*x[0]*0.1 + x[0] + np.random.randn()*1e-3)  #compute output
    x = x[0]/(1.2+x[1]**2) + x[1]*0.4, \
        x[1]/(1.2+x[0]**2) + x[0]*0.4 + uk*(1+x[0]**2/10) #advance state

# Put the inputs and outputs in a Input_output_data format
data = Input_output_data(u=ulist, y=np.array(ylist)) 

# Split dataset
train, val, test  = data[:8000], data[8000:9000], data[9000:]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
nx, nb, na = 2, 5, 5
nu, ny, norm = dsi.get_nu_ny_and_auto_norm(train) 
# Equivilent to 
# nu = 'scalar', ny = 'scalar', norm = dsi.Norm(umean=train.u.mean(0), ustd=train.u.std(0), ymean=train.y.mean(0), ystd=train.y.std(0))
torch.manual_seed(1)
model = dsi.SUBNET(nu, ny, norm, nx=nx, nb=nb, na=na)
# this model has three components:
# print(model.f)
# print(model.h)
# print(model.encoder)

x = torch.randn(1,model.nx) # 1 batch size and nx state vector size 
u = torch.randn(1) #1 batch size and 'scalar' input size
print(f'{model.f(x, u) = }') #the components can be evaluated in desired point
print(f'u: {u.shape}, x: {x.shape}')

### load dataset ###
from nonlinear_benchmarks import Cascaded_Tanks
train_val, test = Cascaded_Tanks()
train, val = train_val[:850], train_val[850:]
print(f'{train_val = }')
print(f'{test = }')

model.f(x, u) = tensor([[ 0.4732, -1.0893]], grad_fn=<AddBackward0>)
u: torch.Size([1]), x: torch.Size([1, 2])
train_val = Input_output_data "train Cascaded_Tanks" u.shape=(1024,) y.shape=(1024,) sampling_time=4.0
test = Input_output_data "test Cascaded_Tanks" u.shape=(1024,) y.shape=(1024,) sampling_time=4.0 state_initialization_window_length=5


In [18]:
train_dict = dsi.fit(model, train=train, val=val, n_its=1000, T=30, batch_size=64, val_freq=100)

Number of samples to train on = 816


  0%|          | 0/1000 [00:00<?, ?it/s]

it       0 NRMS loss nan NRMS val 1.49205!! nan samps/sec
it     100 NRMS loss 0.62930 NRMS val 0.97241!! 3050.25 samps/sec
it     200 NRMS loss 0.30833 NRMS val 0.51499!! 3208.80 samps/sec
it     300 NRMS loss 0.22331 NRMS val 0.57695   3242.12 samps/sec
it     400 NRMS loss 0.21072 NRMS val 0.54340   3254.00 samps/sec
it     500 NRMS loss 0.18998 NRMS val 0.48714!! 3305.23 samps/sec
it     600 NRMS loss 0.18331 NRMS val 0.46515!! 3303.81 samps/sec
it     700 NRMS loss 0.18172 NRMS val 0.48756   3343.17 samps/sec
it     800 NRMS loss 0.16342 NRMS val 0.37120!! 3337.73 samps/sec
it     900 NRMS loss 0.16889 NRMS val 0.37678   3311.22 samps/sec
it   1,000 NRMS loss 0.15464 NRMS val 0.40511   3305.68 samps/sec
